#  クロスワード自動生成

In [1]:
import re

# クロスワードのサイズ
SIZE = 2

# 空白を表す文字
BLANK = '□'

# 使わないマス目を表す文字
UNABLE = '■'



In [2]:
# 辞書を読み込む
# [word_length][initial] = (word, used_flag)
dictionary = dict()

dictionary_file = 'data/test.csv'
# word number
number = 1 
with open(dictionary_file, 'r') as fp:
    for row in fp:
        word = row.strip()
        length = len(word)       
        if length not in dictionary:
            dictionary[length] = dict()
        if word[0] not in dictionary[length]:
            dictionary[length][word[0]] = list()
            
        dictionary[length][word[0]].append((number, word, False))
        number += 1

print(dictionary)

# クロスワードを表す2次元行列
"""
data_list[i][j] => (i+1)行名の(j+1)列目を表す
"""
data_list = list()
# クロスワード初期化
# 処理を簡単にするため一回り大きいサイズのリストにする
for i in range(0, SIZE+2):
    data_list.append([BLANK]*(SIZE+2))
    data_list[i][0] = UNABLE
    data_list[i][SIZE+1] = UNABLE

# 一回り大きいサイズにしたので、外周をすべてunableにする
data_list[0] = [UNABLE]*(SIZE+2)
data_list[SIZE+1] = [UNABLE]*(SIZE+2)
        
# 空白の場所を決める
data_list[2][1] = UNABLE


{2: {'ヤ': [(1, 'ヤマ', False)], 'マ': [(2, 'マリ', False)]}}


In [3]:
def _show_program(data):
    """
    クロスワードをいい感じで表示する
    """
    
    size = len(data)
    
    h_pipe = ''
    v_pipe = ''
    
    print(h_pipe*size)
    
    for i in range(0, size):
        print(v_pipe.join(data_list[i]))
    
    print(h_pipe*size)

In [4]:
def _get_line_list(data):
    """
    必要な縦、横のリストを求める
    A1タテN文字とか、C3ヨコM文字とか…
    """
    v_list = list()
    h_list = list()
    
    size = len(data)
    
    for h in range(0, size):
        for v in range(0, size):
            if data[h][v] == UNABLE:
                continue
            
            # タテ
            if data[h-1][v] == UNABLE and data[h+1][v] != UNABLE:
                for i in range(1, size+1):
                    if data[h+i][v] == UNABLE:
                        v_list.append((h, v, '□'*int(i)))
                        break
            # ヨコ
            if data[h][v-1] == UNABLE and data[h][v+1] != UNABLE:
                for i in range(1, size+1):
                    if data[h][v+i] == UNABLE:
                        h_list.append((h, v, '□'*int(i)))
                        break
                        
    return v_list, h_list

v_answer, h_answer= _get_line_list(data_list)

print(v_answer)
print(h_answer)


[(1, 2, '□□')]
[(1, 1, '□□')]


In [5]:
def _search_word(pattern, dictionary, number):
    """
    辞書からpatternに合った言葉を抜き出す
    言葉が存在しない場合、None
    """
    
    length = len(pattern)
    regex = re.compile(pattern)
    
    if pattern[0] == '.':
        for inital, word_list in dictionary[length].items():
            for i, word in enumerate(word_list):
                if word[0] <= number or word[2]:
                    continue
                if regex.match(word[1]):
                    dictionary[length][inital][i] = (word[0], word[1], True)
                    return word[0], word[1], dictionary
    else:
        for i, word in enumerate(dictionary[length][pattern[0]]):
            if word[0] <= number or word[2]:
                    continue
            if regex.match(word[1]):
                dictionary[length][pattern[0]][i] = (word[0], word[1], True)
                return word[0], word[1], dictionary
    
    return 0, None, dictionary

def _create_corss_word(data, tate_list, yoko_list, index, dictionary):
    """
    クロスワードをつくる
    """
    
    if len(tate_list) <= index and len(yoko_list) <= index:
        return data
    
    # タテに入る言葉を見つける
    if index < len(tate_list):
        # セルの位置と文字数
        h_index, v_index, word = tate_list[index]
        print('tate: %s' % [h_index, v_index, word])
        
        # まだ言葉が決まっていないなら
        if BLANK in word:
            candidate = ''
            for i in range(0, len(word)):
                if data[h_index+i][v_index] == BLANK:
                    candidate += '.'
                else:
                    candidate += data[h_index+i][v_index]
            
            print(candidate)
            
            number = 0
            while True:
                print('tate hit: %s' % [number, word, dictionary])
                
                number, word, new_dictionary = _search_word(candidate, dictionary, number)
                
                print('tate hit: %s' % [number, word, dictionary])
                print('tate hit: %s' % [number, word, new_dictionary])
                
                if number == 0:
                    return None
                
                tate_list[index] = (h_index, v_index, word)
                
                for i in range(0, len(word)):
                    data[h_index+i][v_index] = word[i]

                _show_program(data)

                result = _create_corss_word(data, tate_list, yoko_list, index, new_dictionary)
                
                if result is not None:
                    break
    
    # ヨコに入る言葉を見つける
    if index < len(yoko_list):
        # セルの位置と文字数
        h_index, v_index, word = yoko_list[index]
        print('yoko: %s' % [h_index, v_index, word])
        
        # まだ言葉が決まっていないなら
        if BLANK in word:
            candidate = ''
            for i in range(0, len(word)):
                if data[h_index][v_index+i] == BLANK:
                    candidate += '.'
                else:
                    candidate += data[h_index][v_index+i]
                    
            print(candidate)
            
            number = 0
            while True:  
                number, word, new_dictionary = _search_word(candidate, dictionary, number)
                
                print('yoko hit: %s' % [number, word, dictionary])
                
                if number == 0:
                    return None
            
                _show_program(data)
                
                yoko_list[index] = (h_index, v_index, word)

                for i in range(0, len(word)):
                    data[h_index][v_index+i] = word[i]
           
                result = _create_corss_word(data, tate_list, yoko_list, index, new_dictionary)
                
                if result is not None:
                    break

    return _create_corss_word(data, tate_list, yoko_list, index+1, dictionary)

_show_program(data_list)

# クロスワードを作る
result = _create_corss_word(data_list, v_answer, h_answer, 0, dictionary)

print(result)

_show_program(result)


■■■■
■□□■
■■□■
■■■■

tate: [1, 2, '□□']
..
tate hit: [0, '□□', {2: {'ヤ': [(1, 'ヤマ', False)], 'マ': [(2, 'マリ', False)]}}]
tate hit: [1, 'ヤマ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', False)]}}]
tate hit: [1, 'ヤマ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', False)]}}]

■■■■
■□ヤ■
■■マ■
■■■■

tate: [1, 2, 'ヤマ']
yoko: [1, 1, '□□']
.ヤ
yoko hit: [0, None, {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', False)]}}]
tate hit: [1, 'ヤマ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', False)]}}]
tate hit: [2, 'マリ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]
tate hit: [2, 'マリ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]

■■■■
■□マ■
■■リ■
■■■■

tate: [1, 2, 'マリ']
yoko: [1, 1, '□□']
.マ
yoko hit: [0, None, {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]
tate hit: [2, 'マリ', {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]
tate hit: [0, None, {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]
tate hit: [0, None, {2: {'ヤ': [(1, 'ヤマ', True)], 'マ': [(2, 'マリ', True)]}}]

TypeError: object of type 'NoneType' has no len()